# BentoML Example: Keras Toxic Comment Classification


[BentoML](http://bentoml.ai) is an open source platform for machine learning model serving and deployment. 

This notebook demonstrates how to use BentoML to turn a Keras model into a docker image containing a REST API server serving this model, how to use your ML service built with BentoML as a CLI tool, and how to distribute it a pypi package.


This notebook is built based on: https://www.kaggle.com/sarvajna/keras-sequential-model-lb-0-052

![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=keras&ea=keras-toxic-comment-classification&dt=keras-toxic-comment-classification)

In [4]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
!pip install bentoml
!pip install keras kaggle tensorflow==1.14.0

In [5]:
import bentoml
import numpy as np
import pandas as pd
from keras.preprocessing import text, sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [6]:
list_of_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
max_features = 20000
max_text_length = 400
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
batch_size = 32
epochs = 2

## Prepare Dataset

Please Download data with Kaggle at https://www.kaggle.com/sarvajna/keras-sequential-model-lb-0-052/data

If you are running this notebook in Google Colab, fill in your kaggle credential below and download the training dataset from Kaggle:

In [7]:
%%bash

export KAGGLE_USERNAME=
export KAGGLE_KEY=

if [ ! -f ./train.csv.zip ]; then
    kaggle competitions download -c jigsaw-toxic-comment-classification-challenge
    unzip train.csv.zip
    unzip sample_submission.csv.zip
    unzip test.csv.zip
    unzip test_labels.csv.zip
fi

In [8]:
train_df = pd.read_csv('./train.csv')

print(train_df.head())

                 id                                       comment_text  toxic  \
0  0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   
1  000103f0d9cfb60f  D'aww! He matches this background colour I'm s...      0   
2  000113f07ec002fd  Hey man, I'm really not trying to edit war. It...      0   
3  0001b41b1c6bb37e  "\nMore\nI can't make any real suggestions on ...      0   
4  0001d958c54c6e35  You, sir, are my hero. Any chance you remember...      0   

   severe_toxic  obscene  threat  insult  identity_hate  
0             0        0       0       0              0  
1             0        0       0       0              0  
2             0        0       0       0              0  
3             0        0       0       0              0  
4             0        0       0       0              0  


In [9]:
x = train_df['comment_text'].values
print(x)

["Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"
 "D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)"
 "Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info."
 ...
 'Spitzer \n\nUmm, theres no actual article for prostitution ring.  - Crunch Captain.'
 'And it looks like it was actually you who put on the speedy to have the first version deleted now that I look at it.'
 '"\nAnd ... I really don\'t think you understand.  I came here and my idea was bad right away.  What kind of community goes ""you have bad ideas"" go away, instead o

In [10]:
y = train_df[list_of_classes].values
print(y)

[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 ...
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]


In [11]:
x_tokenizer = text.Tokenizer(num_words=max_features)
print(x_tokenizer)
x_tokenizer.fit_on_texts(list(x))
print(x_tokenizer)
x_tokenized = x_tokenizer.texts_to_sequences(x) #list of lists(containing numbers), so basically a list of sequences, not a numpy array
#pad_sequences:transform a list of num_samples sequences (lists of scalars) into a 2D Numpy array of shape 
x_train_val = sequence.pad_sequences(x_tokenized, maxlen=max_text_length)

In [12]:
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y, test_size=0.1, random_state=1)

In [13]:
print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=max_text_length))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto 6 output layers, and squash it with a sigmoid:
model.add(Dense(6))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

W0213 14:05:45.043107 4456553920 deprecation_wrapper.py:119] From /usr/local/anaconda3/envs/dev-py3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0213 14:05:45.058714 4456553920 deprecation_wrapper.py:119] From /usr/local/anaconda3/envs/dev-py3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0213 14:05:45.061474 4456553920 deprecation_wrapper.py:119] From /usr/local/anaconda3/envs/dev-py3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0213 14:05:45.076580 4456553920 deprecation_wrapper.py:119] From /usr/local/anaconda3/envs/dev-py3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please us

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 400, 50)           1000000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 400, 50)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 398, 250)          37750     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 250)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 250)               62750     
_________________________________________________________________
dropout_2 (Dropout)          (None, 250)               0         
_________________________________________________________________
activation_1 (Activation)    (None, 250)               0     

In [14]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
validation_data=(x_val, y_val))

Train on 143613 samples, validate on 15958 samples
Epoch 1/2
143613/143613 [==============================] - 248s 2ms/step - loss: 0.0631 - acc: 0.9787 - val_loss: 0.0529 - val_acc: 0.9808
Epoch 2/2
143613/143613 [==============================] - 250s 2ms/step - loss: 0.0463 - acc: 0.9828 - val_loss: 0.0509 - val_acc: 0.9812


In [15]:
test_df = pd.read_csv('./test.csv')

In [16]:
x_test = test_df['comment_text'].values

In [17]:
x_test_tokenized = x_tokenizer.texts_to_sequences(x_test)
x_testing = sequence.pad_sequences(x_test_tokenized, maxlen=max_text_length)

In [18]:
y_testing = model.predict(x_testing, verbose = 1)

153164/153164 [==============================] - 43s 283us/step


In [19]:
sample_submission = pd.read_csv("./sample_submission.csv")
sample_submission[list_of_classes] = y_testing
sample_submission.to_csv("toxic_comment_classification.csv", index=False)

## Create BentoService for model serving

In [20]:
%%writefile toxic_comment_classifier.py

from bentoml import api, artifacts, env, BentoService
from bentoml.artifact import PickleArtifact, KerasModelArtifact
from bentoml.adapters import DataframeInput

from keras.preprocessing import text, sequence
import numpy as np

list_of_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
max_text_length = 400

@env(pip_dependencies=['tensorflow==1.14.0', 'keras', 'pandas', 'numpy'])
@artifacts([PickleArtifact('x_tokenizer'), KerasModelArtifact('model')])
class ToxicCommentClassification(BentoService):
    
    def tokenize_df(self, df):
        comments = df['comment_text'].values
        tokenized = self.artifacts.x_tokenizer.texts_to_sequences(comments)        
        input_data = sequence.pad_sequences(tokenized, maxlen=max_text_length)
        return input_data
    
    @api(input=DataframeInput())
    def predict(self, df):
        input_data = self.tokenize_df(df)
        prediction = self.artifacts.model.predict(input_data)
        result = []
        for i in prediction:
            result.append(list_of_classes[np.argmax(i)])
        return result

Overwriting toxic_comment_classifier.py


## Save BentoService to file archive

In [21]:
# 1) import the custom BentoService defined above
from toxic_comment_classifier import ToxicCommentClassification

# 2) `pack` it with required artifacts
svc = ToxicCommentClassification()
svc.pack('x_tokenizer', x_tokenizer)
svc.pack('model', model)

# 3) save your BentoSerivce
saved_path = svc.save()

[2020-02-13 14:15:03,839] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.6.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'


W0213 14:15:03.842620 4456553920 deprecation_wrapper.py:119] From /Users/bozhaoyu/src/bento/bentoml/artifact/keras_model_artifact.py:100: The name tf.keras.backend.get_session is deprecated. Please use tf.compat.v1.keras.backend.get_session instead.



[2020-02-13 14:15:04,172] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.6.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-02-13 14:15:35,253] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.6.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml

no previously-included directories found matching 'examples'
no previously-included directories found matching 'tests'
no previously-included directories found matching 'docs'
no previously-included directories found matching 'scripts'


writing manifest file 'BentoML.egg-info/SOURCES.txt'
running check


creating BentoML-0.6.2+8.gd95a887
creating BentoML-0.6.2+8.gd95a887/BentoML.egg-info
creating BentoML-0.6.2+8.gd95a887/bentoml
creating BentoML-0.6.2+8.gd95a887/bentoml/artifact
creating BentoML-0.6.2+8.gd95a887/bentoml/bundler
creating BentoML-0.6.2+8.gd95a887/bentoml/cli
creating BentoML-0.6.2+8.gd95a887/bentoml/clipper
creating BentoML-0.6.2+8.gd95a887/bentoml/configuration
creating BentoML-0.6.2+8.gd95a887/bentoml/deployment
creating BentoML-0.6.2+8.gd95a887/bentoml/deployment/aws_lambda
creating BentoML-0.6.2+8.gd95a887/bentoml/deployment/sagemaker
creating BentoML-0.6.2+8.gd95a887/bentoml/handlers
creating BentoML-0.6.2+8.gd95a887/bentoml/marshal
creating BentoML-0.6.2+8.gd95a887/bentoml/migrations
creating BentoML-0.6.2+8.gd95a887/bentoml/migrations/versions
creating BentoML-0.6.2+8.gd95a887/bentoml/proto
creating BentoML-0.6.2+8.gd95a887/bentoml/repository
creating BentoML-0.6.2+8.gd95a887/bentoml/server
creating BentoML-0.6.2+8.gd95a887/bentoml/server/static
creating BentoML-0

copying bentoml/proto/yatai_service_pb2.py -> BentoML-0.6.2+8.gd95a887/bentoml/proto
copying bentoml/proto/yatai_service_pb2_grpc.py -> BentoML-0.6.2+8.gd95a887/bentoml/proto
copying bentoml/repository/__init__.py -> BentoML-0.6.2+8.gd95a887/bentoml/repository
copying bentoml/repository/metadata_store.py -> BentoML-0.6.2+8.gd95a887/bentoml/repository
copying bentoml/server/__init__.py -> BentoML-0.6.2+8.gd95a887/bentoml/server
copying bentoml/server/bento_api_server.py -> BentoML-0.6.2+8.gd95a887/bentoml/server
copying bentoml/server/bento_sagemaker_server.py -> BentoML-0.6.2+8.gd95a887/bentoml/server
copying bentoml/server/gunicorn_config.py -> BentoML-0.6.2+8.gd95a887/bentoml/server
copying bentoml/server/gunicorn_server.py -> BentoML-0.6.2+8.gd95a887/bentoml/server
copying bentoml/server/marshal_server.py -> BentoML-0.6.2+8.gd95a887/bentoml/server
copying bentoml/server/middlewares.py -> BentoML-0.6.2+8.gd95a887/bentoml/server
copying bentoml/server/utils.py -> BentoML-0.6.2+8.gd95a

## Load BentoService from archive

In [22]:
sample_test = test_df.iloc[40:42]
print(sample_test)
bento_service = bentoml.load(saved_path)

print(bento_service.predict(sample_test))

                  id                                       comment_text
40  0011cefc680993ba                      REDIRECT Talk:Mi Vida Eres Tú
41  0011ef6aa33d42e6  " \n I'm not convinced that he was blind. Wher...
[2020-02-13 14:15:54,065] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.6.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-02-13 14:15:54,084] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.6.2,  but loading from BentoML version 0.6.2+8.gd95a887
[2020-02-13 

W0213 14:15:54.356201 4456553920 deprecation_wrapper.py:119] From /Users/bozhaoyu/src/bento/bentoml/artifact/keras_model_artifact.py:114: The name tf.keras.backend.set_session is deprecated. Please use tf.compat.v1.keras.backend.set_session instead.



[2020-02-13 14:15:56,008] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.6.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
['toxic', 'toxic']


In [23]:
!bentoml get ToxicCommentClassification

BENTO_SERVICE                                     AGE            APIS                       ARTIFACTS
ToxicCommentClassification:20200213141504_137C94  26.84 seconds  predict<DataframeHandler>  x_tokenizer<PickleArtifact>, model<KerasModelArtifact>


In [24]:
!bentoml get ToxicCommentClassification:20200213141504_137C94 

{
  "name": "ToxicCommentClassification",
  "version": "20200213141504_137C94",
  "uri": {
    "type": "LOCAL",
    "uri": "/Users/bozhaoyu/bentoml/repository/ToxicCommentClassification/20200213141504_137C94"
  },
  "bentoServiceMetadata": {
    "name": "ToxicCommentClassification",
    "version": "20200213141504_137C94",
    "createdAt": "2020-02-13T22:15:35.254603Z",
    "env": {
      "condaEnv": "name: bentoml-ToxicCommentClassification\nchannels:\n- defaults\ndependencies:\n- python=3.7.3\n- pip\n",
      "pipDependencies": "bentoml==0.6.2\ntensorflow==1.14.0\nkeras\npandas\nnumpy",
      "pythonVersion": "3.7.3"
    },
    "artifacts": [
      {
        "name": "x_tokenizer",
        "artifactType": "PickleArtifact"
      },
      {
        "name": "model",
        "artifactType": "KerasModelArtifact"
      }
    ],
    "apis": [
      {
        "name": "predict",
        "handlerType": "DataframeHandler",
        "docs": "BentoService API",
        "handlerConfig": {
          "

In [25]:
!bentoml info ToxicCommentClassification:20200213141504_137C94 

[2020-02-13 14:16:23,638] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.6.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-02-13 14:16:23,664] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.6.2,  but loading from BentoML version 0.6.2+8.gd95a887
{
  "name": "ToxicCommentClassification",
  "version": "20200213141504_137C94",
  "created_at": "2020-02-13T22:15:35.254603Z",
  "env": {
    "conda_env": "name: bentoml-ToxicCommentClassification\nchannels:\n- defaults\ndepende

In [26]:
!bentoml run ToxicCommentClassification:20200213141504_137C94 predict --input '[{"comment_text": "bad terrible"}]'

[2020-02-13 14:16:34,090] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.6.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-02-13 14:16:34,112] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.6.2,  but loading from BentoML version 0.6.2+8.gd95a887
Using TensorFlow backend.
[2020-02-13 14:16:37,621] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the

## Use BentoService as PyPI package

In [29]:
!pip install {saved_path}

Processing /Users/bozhaoyu/bentoml/repository/ToxicCommentClassification/20200213131414_B4566D


  Created wheel for ToxicCommentClassification: filename=ToxicCommentClassification-20200213131414_B4566D-py3-none-any.whl size=16121004 sha256=69af4300684b8598686bcf4707c5a5044d52425ac2e671ed1542981c2741312c
  Stored in directory: /private/var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/pip-ephem-wheel-cache-8ycwo0oi/wheels/ef/00/db/6c76f4e5d4f074deccf533c31c3bb255c8f5cb43809457dffd
Successfully built ToxicCommentClassification


In [30]:
import ToxicCommentClassification

svc = ToxicCommentClassification.load()
result = svc.predict(sample_test)
result

['toxic', 'toxic']

## Deploy BentoService as REST API server to the cloud


BentoML support deployment to multiply cloud provider services, such as AWS Lambda, AWS Sagemaker, Google Cloudrun and etc. You can find the full list and guide on the documentation site at https://docs.bentoml.org/en/latest/deployment/index.html

For this project, we are going to deploy to AWS Sagemaker

**Use `bentoml sagemaker deploy` to deploy BentoService to AWS Sagemaker**

In [28]:
!bentoml sagemaker deploy keras-toxic -b ToxicCommentClassification:20200213141504_137C94 \
    --api-name predict --verbose

[2020-02-13 14:17:10,940] DEBUG - Using BentoML with local Yatai server
[2020-02-13 14:17:11,041] DEBUG - Upgrading tables to the latest revision
Deploying Sagemaker deployment \[2020-02-13 14:17:11,841] DEBUG - Created temporary directory: /private/var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/bentoml-temp-mblyir49
\[2020-02-13 14:17:12,274] DEBUG - Getting docker login info from AWS
[2020-02-13 14:17:12,275] DEBUG - Building docker image: 192023623294.dkr.ecr.us-west-2.amazonaws.com/toxiccommentclassification-sagemaker:20200213141504_137C94
-[2020-02-13 14:17:13,496] INFO - Step 1/11 : FROM continuumio/miniconda3:4.7.12
[2020-02-13 14:17:13,497] INFO - 

[2020-02-13 14:17:13,497] INFO -  ---> 406f2b43ea59

[2020-02-13 14:17:13,497] INFO - Step 2/11 : EXPOSE 8080
[2020-02-13 14:17:13,498] INFO - 

[2020-02-13 14:17:13,498] INFO -  ---> Using cache

[2020-02-13 14:17:13,498] INFO -  ---> 85b4bb5fff81

[2020-02-13 14:17:13,498] INFO - Step 3/11 : RUN set -x      && apt-get update   

[2020-02-13 14:18:27,839] INFO -   Downloading SQLAlchemy-1.3.13.tar.gz (6.0 MB)

\[2020-02-13 14:18:30,884] INFO - Collecting cerberus

[2020-02-13 14:18:30,900] INFO -   Downloading Cerberus-1.3.2.tar.gz (52 kB)

\[2020-02-13 14:18:31,249] INFO - Collecting python-json-logger

[2020-02-13 14:18:31,265] INFO -   Downloading python-json-logger-0.1.11.tar.gz (6.0 kB)

|[2020-02-13 14:18:31,579] INFO - Collecting tabulate

[2020-02-13 14:18:31,590] INFO -   Downloading tabulate-0.8.6.tar.gz (45 kB)

|[2020-02-13 14:18:31,941] INFO - Collecting configparser

[2020-02-13 14:18:31,953] INFO -   Downloading configparser-4.0.2-py2.py3-none-any.whl (22 kB)

-[2020-02-13 14:18:32,168] INFO - Collecting protobuf>=3.6.0

[2020-02-13 14:18:32,181] INFO -   Downloading protobuf-3.11.3-cp37-cp37m-manylinux1_x86_64.whl (1.3 MB)

\[2020-02-13 14:18:32,483] INFO - Requirement already satisfied: requests in /opt/conda/lib/python3.7/site-packages (from bentoml==0.6.2->-r /opt/program/requirements.t

-[2020-02-13 14:18:46,103] INFO -   Building wheel for sqlalchemy (setup.py): finished with status 'done'

[2020-02-13 14:18:46,108] INFO -   Created wheel for sqlalchemy: filename=SQLAlchemy-1.3.13-cp37-cp37m-linux_x86_64.whl size=1223705 sha256=96f2931d8d4aab0357a906f7d2886db52ef463f7d5c658db467aac5148db398f
  Stored in directory: /root/.cache/pip/wheels/b9/ba/77/163f10f14bd489351530603e750c195b0ceceed2f3be2b32f1

[2020-02-13 14:18:46,110] INFO -   Building wheel for cerberus (setup.py): started

\[2020-02-13 14:18:46,440] INFO -   Building wheel for cerberus (setup.py): finished with status 'done'

[2020-02-13 14:18:46,441] INFO -   Created wheel for cerberus: filename=Cerberus-1.3.2-py3-none-any.whl size=54335 sha256=950cf9bdaff98c4c9c442537d2f077af382916bc35e0556860dc2160e2740999

[2020-02-13 14:18:46,442] INFO -   Stored in directory: /root/.cache/pip/wheels/17/3a/0d/e2fc48cf85cb858f5e65f1baa36180ebb5dce6397c35c4cfcb

[2020-02-13 14:18:46,444] INFO -   Building wheel for python

-[2020-02-13 14:21:13,199] DEBUG - AWS create endpoint config response: {'EndpointConfigArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint-config/bobo-keras-toxic-toxiccommentclassifi-20200213141504-137c94', 'ResponseMetadata': {'RequestId': 'd9c178b6-cb46-47ac-9f02-f87d0686f860', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'd9c178b6-cb46-47ac-9f02-f87d0686f860', 'content-type': 'application/x-amz-json-1.1', 'content-length': '140', 'date': 'Thu, 13 Feb 2020 22:21:12 GMT'}, 'RetryAttempts': 0}}
[2020-02-13 14:21:13,200] DEBUG - Creating sagemaker endpoint bobo-keras-toxic
|[2020-02-13 14:21:13,401] DEBUG - AWS create endpoint response: {'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-keras-toxic', 'ResponseMetadata': {'RequestId': '5b68def6-2df7-4515-8a22-3efa2a0cf20e', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '5b68def6-2df7-4515-8a22-3efa2a0cf20e', 'content-type': 'application/x-amz-json-1.1', 'content-length': '84', 'dat

-[2020-02-13 14:22:00,320] DEBUG - AWS describe endpoint response: {'EndpointName': 'bobo-keras-toxic', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-keras-toxic', 'EndpointConfigName': 'bobo-keras-toxic-ToxicCommentClassifi-20200213141504-137C94', 'EndpointStatus': 'Creating', 'CreationTime': datetime.datetime(2020, 2, 13, 14, 21, 13, 367000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2020, 2, 13, 14, 21, 13, 367000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': '5b57327b-b2ce-4ca8-b4cd-c0ca0277bc84', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '5b57327b-b2ce-4ca8-b4cd-c0ca0277bc84', 'content-type': 'application/x-amz-json-1.1', 'content-length': '297', 'date': 'Thu, 13 Feb 2020 22:21:59 GMT'}, 'RetryAttempts': 0}}
-[2020-02-13 14:22:05,668] DEBUG - AWS describe endpoint response: {'EndpointName': 'bobo-keras-toxic', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-keras-toxic', 'EndpointConfigName': 

...

-[2020-02-13 14:24:53,505] DEBUG - AWS describe endpoint response: {'EndpointName': 'bobo-keras-toxic', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-keras-toxic', 'EndpointConfigName': 'bobo-keras-toxic-ToxicCommentClassifi-20200213141504-137C94', 'EndpointStatus': 'Creating', 'CreationTime': datetime.datetime(2020, 2, 13, 14, 21, 13, 367000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2020, 2, 13, 14, 21, 13, 367000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': '4c015c57-d57a-4f0e-b7fa-7ce8d957be66', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '4c015c57-d57a-4f0e-b7fa-7ce8d957be66', 'content-type': 'application/x-amz-json-1.1', 'content-length': '297', 'date': 'Thu, 13 Feb 2020 22:24:53 GMT'}, 'RetryAttempts': 0}}
|[2020-02-13 14:24:58,689] DEBUG - AWS describe endpoint response: {'EndpointName': 'bobo-keras-toxic', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-keras-toxic', 'EndpointConfigName': 

/[2020-02-13 14:25:50,855] DEBUG - AWS describe endpoint response: {'EndpointName': 'bobo-keras-toxic', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-keras-toxic', 'EndpointConfigName': 'bobo-keras-toxic-ToxicCommentClassifi-20200213141504-137C94', 'EndpointStatus': 'Creating', 'CreationTime': datetime.datetime(2020, 2, 13, 14, 21, 13, 367000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2020, 2, 13, 14, 21, 13, 367000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': '0495a2bf-a496-4f2e-9174-41b2d646a1f9', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '0495a2bf-a496-4f2e-9174-41b2d646a1f9', 'content-type': 'application/x-amz-json-1.1', 'content-length': '297', 'date': 'Thu, 13 Feb 2020 22:25:50 GMT'}, 'RetryAttempts': 0}}
-[2020-02-13 14:25:56,043] DEBUG - AWS describe endpoint response: {'EndpointName': 'bobo-keras-toxic', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-keras-toxic', 'EndpointConfigName': 

\[2020-02-13 14:26:47,989] DEBUG - AWS describe endpoint response: {'EndpointName': 'bobo-keras-toxic', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-keras-toxic', 'EndpointConfigName': 'bobo-keras-toxic-ToxicCommentClassifi-20200213141504-137C94', 'EndpointStatus': 'Creating', 'CreationTime': datetime.datetime(2020, 2, 13, 14, 21, 13, 367000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2020, 2, 13, 14, 21, 13, 367000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': '989fb689-7645-4fa9-b30c-b375780911d4', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '989fb689-7645-4fa9-b30c-b375780911d4', 'content-type': 'application/x-amz-json-1.1', 'content-length': '297', 'date': 'Thu, 13 Feb 2020 22:26:47 GMT'}, 'RetryAttempts': 0}}
|[2020-02-13 14:26:53,214] DEBUG - AWS describe endpoint response: {'EndpointName': 'bobo-keras-toxic', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-keras-toxic', 'EndpointConfigName': 

|[2020-02-13 14:27:45,064] DEBUG - AWS describe endpoint response: {'EndpointName': 'bobo-keras-toxic', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-keras-toxic', 'EndpointConfigName': 'bobo-keras-toxic-ToxicCommentClassifi-20200213141504-137C94', 'EndpointStatus': 'Creating', 'CreationTime': datetime.datetime(2020, 2, 13, 14, 21, 13, 367000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2020, 2, 13, 14, 21, 13, 367000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': 'cf7dcc62-8e7b-4526-b525-7a05fa78de37', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'cf7dcc62-8e7b-4526-b525-7a05fa78de37', 'content-type': 'application/x-amz-json-1.1', 'content-length': '297', 'date': 'Thu, 13 Feb 2020 22:27:44 GMT'}, 'RetryAttempts': 0}}
/[2020-02-13 14:27:50,599] DEBUG - AWS describe endpoint response: {'EndpointName': 'bobo-keras-toxic', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-keras-toxic', 'EndpointConfigName': 

\[2020-02-13 14:28:42,427] DEBUG - AWS describe endpoint response: {'EndpointName': 'bobo-keras-toxic', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-keras-toxic', 'EndpointConfigName': 'bobo-keras-toxic-ToxicCommentClassifi-20200213141504-137C94', 'EndpointStatus': 'Creating', 'CreationTime': datetime.datetime(2020, 2, 13, 14, 21, 13, 367000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2020, 2, 13, 14, 21, 13, 367000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': 'd37cf3bc-694b-48b3-953c-cc3e49525b33', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'd37cf3bc-694b-48b3-953c-cc3e49525b33', 'content-type': 'application/x-amz-json-1.1', 'content-length': '297', 'date': 'Thu, 13 Feb 2020 22:28:41 GMT'}, 'RetryAttempts': 0}}
|[2020-02-13 14:28:47,608] DEBUG - AWS describe endpoint response: {'EndpointName': 'bobo-keras-toxic', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-keras-toxic', 'EndpointConfigName': 

`bentoml sagemaker list` displays all deployed Sagemaker deployments

In [29]:
!bentoml sagemaker list

NAME         NAMESPACE    PLATFORM       BENTO_SERVICE                                     STATUS    AGE
keras-toxic  bobo         aws-sagemaker  ToxicCommentClassification:20200213141504_137C94  running   12 minutes and 44.12 seconds


`bentoml sagemaker get` retrieve the latest status of Sagemaker deployment

In [30]:
!bentoml sagemaker get keras-toxic

{
  "namespace": "bobo",
  "name": "keras-toxic",
  "spec": {
    "bentoName": "ToxicCommentClassification",
    "bentoVersion": "20200213141504_137C94",
    "operator": "AWS_SAGEMAKER",
    "sagemakerOperatorConfig": {
      "region": "us-west-2",
      "instanceType": "ml.m4.xlarge",
      "instanceCount": 1,
      "apiName": "predict"
    }
  },
  "state": {
    "state": "RUNNING",
    "infoJson": {
      "EndpointName": "bobo-keras-toxic",
      "EndpointArn": "arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-keras-toxic",
      "EndpointConfigName": "bobo-keras-toxic-ToxicCommentClassifi-20200213141504-137C94",
      "ProductionVariants": [
        {
          "VariantName": "bobo-keras-toxic-ToxicCommentClassifi-20200213141504-137C94",
          "DeployedImages": [
            {
              "SpecifiedImage": "192023623294.dkr.ecr.us-west-2.amazonaws.com/toxiccommentclassification-sagemaker:20200213141504_137C94",
              "ResolvedImage": "192023623294.dkr.ecr.us-wes

Validate and test Sagemaker deployment with sample data

In [31]:
!aws sagemaker-runtime invoke-endpoint --endpoint-name bobo-keras-toxic \
--body '[{"comment_text": "bad terrible"}]' --content-type application/json output.json && cat output.json

{
    "ContentType": "application/json",
    "InvokedProductionVariant": "bobo-keras-toxic-ToxicCommentClassifi-20200213141504-137C94"
}
["toxic"]

`bentoml sagemaker delete` will remove Sagmaker deployment and related resources

In [32]:
!bentoml sagemaker delete keras-toxic

Successfully deleted AWS Sagemaker deployment "keras-toxic"
